In [47]:
import json
import pandas as pd
from difflib import SequenceMatcher
import numpy as np
import sys
import csv

all_matches = pd.read_csv("match_data.csv")

fifa_17_data = pd.read_csv("Player_Data/FIFA_Data/player_data/fifa_17_data_scaled.csv")
fifa_18_data = pd.read_csv("Player_Data/FIFA_Data/player_data/fifa_18_data_scaled.csv")
fifa_19_data = pd.read_csv("Player_Data/FIFA_Data/player_data/fifa_19_data_scaled.csv")
fifa_20_data = pd.read_csv("Player_Data/FIFA_Data/player_data/fifa_20_data_scaled.csv")

fifa_average_data = pd.concat([fifa_17_data, fifa_18_data, fifa_19_data, fifa_20_data])
fifa_average_data = fifa_average_data.groupby('Name').mean().reset_index()
fifa_average_data.to_csv("Player_Data/FIFA_Data/player_data/fifa_average_data_scaled.csv", index=False)

fifa_data = {
    "2017" : fifa_17_data,
    "2018" : fifa_18_data,
    "2019" : fifa_19_data,
    "2020" : fifa_20_data,
    "avg" : fifa_average_data
}

In [122]:
pl_xg = pd.read_csv("pl_xg_data.csv")
combined = pd.merge(pl_xg[['season', 'home_team', 'away_team', 'home_spi', 'away_spi', 'home_xg', 'away_xg']], all_matches, on=["season", "home_team", "away_team"])
combined['season'] = combined['season'].astype(str)
combined.to_csv("match_data_combined.csv", index=False)

In [171]:
def getPlayerData(playername,cols,year_data):
    data = []
    if len(playername) == 2:
            playername = playername[0]
    if year_data.loc[year_data['Name'] == playername].empty:
        data = [str(fifa_average_data.loc[fifa_average_data['Name'] == playername,:][col].values[0]) for col in cols]
    else:
        data = [str(year_data.loc[year_data['Name'] == playername,:][col].values[0]) for col in cols]        
    fixed_data = [float(val) for val in data]
    return fixed_data

def findAndFixMissingNames(players,year_data):
    missing = []
    for p in players:
        player_string = p#.encode('ascii', 'ignore')
        if year_data.loc[year_data['Name'] == player_string].empty and fifa_average_data.loc[fifa_average_data['Name'] == player_string].empty:
            if player_string not in allMissing:
                allMissing.append(player_string)
            missing.append(player_string)
    return missing

valid = 0
invalid = 0

def allPlayersFound(lineup,year_data):
    if len(findAndFixMissingNames(lineup,year_data)) > 0:
        return False
    else:
        return True

def format_player_vals(season, players, attrs):
    players = ast.literal_eval(players)
    if allPlayersFound(players, fifa_data[season]):
        return np.array([getPlayerData(player_name,attrs,fifa_data[season]) for player_name in players[:11]]).flatten()
    else:
        return []

In [35]:
def getJSON(filename):
    with open(f"matches/{filename}", 'r') as fp:
        return json.load(fp)

matches = []
for f in ["matches_epl_2019.json"]:
        matches += getJSON(f)

total = len(matches)
missing = 0
allMissing = []
for match in matches:
        findAndFixMissingNames(match['home_lineup'],fifa_19_data)
        findAndFixMissingNames(match['away_lineup'],fifa_19_data)
for missing in allMissing:
        print(missing)
print(len(allMissing))
# fifa_average_data.loc[fifa_average_data['Name'] == 'Ramadan Sobhi'].empty

Yan Valery
Harvey Elliot
Joseph Willock
Sol Bamba
Joe Tupper
Max Sanders
Timothy Eyoma
7


In [172]:
gk_attrs = ['Overall']
def_attrs = ['Overall']
mid_attrs = ['Overall']
str_attrs = ['Overall']

matches = pd.read_csv("match_data_combined.csv")
matches['season'] = matches['season'].astype(str)
matches['home_attrs'] = matches.apply(lambda x: format_player_vals(x.season, x.home_lineup, def_attrs), axis=1)
matches['away_attrs'] = matches.apply(lambda x: format_player_vals(x.season, x.away_lineup, def_attrs), axis=1)

matches.head()

,season,home_team,away_team,home_spi,away_spi,home_xg,away_xg,away_formation,draw_odds,home_formation,home_odds,away_odds,away_lineup,home_score,away_score,home_lineup,home_attrs,away_attrs
0,2017,Hull City,Leicester,53.57,66.81,0.85,2.77,4-4-2,0.275,4-3-3,0.216,0.510,"['Kasper Schmeichel', 'Christian Fuchs', 'Wes ...",2,1,"['Eldin Jakupovic', 'Ahmed Elmohamady', 'Curti...","[0.5412833104680462, 1.2472177648209215, 1.529...","[2.3767128917855227, 1.529591546562072, 1.5295..."
1,2017,Everton,Tottenham,68.02,73.25,0.73,1.11,4-2-3-1,0.284,4-3-3,0.304,0.412,"['Hugo Lloris', 'Danny Rose', 'Toby Alderweire...",1,1,"['Maarten Stekelenburg', 'Mason Holgate', 'Ram...","[1.3884046556914968, 0.1177226378563209, 1.529...","[3.082647346138398, 2.0943391100443725, 2.6590..."
2,2017,Burnley,Swansea,58.98,59.74,1.24,1.84,4-3-3,0.291,4-4-2,0.408,0.301,"['Lukasz Fabianski', 'Kyle Naughton', 'Jordi A...",0,1,"['Tom Heaton', 'Matt Lowton', 'Ben Mee', 'Mich...","[1.8119653283032224, 1.2472177648209215, 1.106...","[1.953152219173797, 1.2472177648209215, 1.5295..."
3,2017,Southampton,Watford,69.49,59.33,1.05,0.22,3-5-2,0.262,4-1-2-1-2,0.544,0.194,"['Heurelho Gomes', 'Craig Cathcart', 'Sebastia...",1,1,"['Fraser Forster', 'Cedric Soares', 'Virgil va...","[1.670778437432647, 1.8119653283032224, 2.2355...","[1.670778437432647, 0.9648439830797716, 1.5295..."
4,2017,Crystal Palace,West Bromwich,55.19,58.66,1.11,0.68,4-4-2,0.294,4-2-3-1,0.490,0.216,"['Ben Foster', 'Jonny Evans', 'Jonas Olsson', ...",0,1,"['Wayne Hennessey', 'Joel Ward', 'Damien Delan...","[1.2472177648209215, 1.2472177648209215, 1.106...","[1.953152219173797, 1.8119653283032224, 1.3884..."


In [217]:
matches['xg_diff'] = matches['home_xg'] - matches['away_xg']
matches['score_diff'] = matches['home_score'] - matches['away_score']
matches['av_home_rating'] = matches['home_attrs'].apply(lambda x: np.mean(x))
matches['av_away_rating'] = matches['away_attrs'].apply(lambda x: np.mean(x))
matches['rating_diff'] = matches['av_home_rating'] - matches['av_away_rating']
matches['result'] = matches[['home_score', 'away_score']].apply(lambda x: 0 if x['home_score'] > x['away_score'] else 1 if x['home_score'] == x['away_score'] else 2,axis=1)

matches['home_length'] = matches['home_attrs'].apply(lambda x: len(x))
matches['away_length'] = matches['away_attrs'].apply(lambda x: len(x))
matches['features_found'] = matches[['home_attrs', 'away_attrs']].apply(lambda x: len(x['home_attrs']) == 11 and len(x['away_attrs']) == 11, axis=1)

matches[['xg_diff', 'score_diff', 'rating_diff']].corr()
matches.head()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


,season,home_team,away_team,home_spi,away_spi,home_xg,away_xg,away_formation,draw_odds,home_formation,...,away_attrs,xg_diff,score_diff,av_home_rating,av_away_rating,rating_diff,home_length,away_length,features_found,result
0,2017,Hull City,Leicester,53.57,66.81,0.85,2.77,4-4-2,0.275,4-3-3,...,"[2.3767128917855227, 1.529591546562072, 1.5295...",-1.92,1,1.067525,1.606603,-0.539077,11,11,True,0
1,2017,Everton,Tottenham,68.02,73.25,0.73,1.11,4-2-3-1,0.284,4-3-3,...,"[3.082647346138398, 2.0943391100443725, 2.6590...",-0.38,0,1.823051,2.338207,-0.515156,11,11,True,1
2,2017,Burnley,Swansea,58.98,59.74,1.24,1.84,4-3-3,0.291,4-4-2,...,"[1.953152219173797, 1.2472177648209215, 1.5295...",-0.60,-1,1.131701,1.465416,-0.333714,11,11,True,2
3,2017,Southampton,Watford,69.49,59.33,1.05,0.22,3-5-2,0.262,4-1-2-1-2,...,"[1.670778437432647, 0.9648439830797716, 1.5295...",0.83,0,1.452581,1.467506,-0.014926,11,11,True,1
4,2017,Crystal Palace,West Bromwich,55.19,58.66,1.11,0.68,4-4-2,0.294,4-2-3-1,...,"[1.953152219173797, 1.8119653283032224, 1.3884...",0.43,-1,1.362734,1.439745,-0.077011,11,11,True,2


In [233]:
input_matches = matches[matches['features_found'] == True]

train_features = input_matches[~(input_matches['season'] == "2020")][['home_attrs', 'away_attrs']].values
train_targets = input_matches[~(input_matches['season'] == "2020")]['xg_diff'].values.astype(float)
test_features = input_matches[input_matches['season'] == "2020"][['home_attrs', 'away_attrs']].values
test_targets = input_matches[input_matches['season'] == "2020"]['xg_diff'].values.astype(float)
test_results = input_matches[input_matches['season'] == "2020"]['result'].values
test_odds = input_matches[input_matches['season'] == "2020"][['home_odds', 'draw_odds', 'away_odds']].values


train_features = np.array([np.concatenate((feat[0], feat[1])) for feat in train_features])
test_features = np.array([np.concatenate((feat[0], feat[1])) for feat in test_features])

print(len(train_features), len(test_features))

1041 242


In [237]:
import tensorflow as tf
import tensorflow_probability as tfp


def run_experiment(model, loss, train_features, train_targets, test_features, test_targets):

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
        loss=loss,
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(x=train_features, y=train_targets, epochs=150, validation_data=(test_features, test_targets), verbose=False)
    
    print("Model training finished.")
    _, rmse = model.evaluate(x=train_features, y=train_targets, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(x=test_features, y=test_targets, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


def create_model(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Dense(64)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.Dense(32)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    distribution_params = tf.keras.layers.Dense(units=2)(x)
    output = tfp.layers.IndependentNormal(1)(distribution_params)

    return tf.keras.Model(inputs,output)

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


model = create_model(len(train_features[0]))
run_experiment(model, negative_loglikelihood, train_features, train_targets, test_features, test_targets)


Start training the model...
Model training finished.
Train RMSE: 0.701
Evaluating model performance...
Test RMSE: 1.444


In [238]:
prediction_distribution = model(test_features)

preds = np.array([1 - prediction_distribution.cdf(0.25), prediction_distribution.cdf(0.25) - prediction_distribution.cdf(-0.25),prediction_distribution.cdf(-0.25)])
preds = preds.transpose()
print(np.mean(np.argmax(preds, axis=1) == test_results), np.mean(np.argmax(test_odds, axis=1) == test_results))

0.4834710743801653 0.5247933884297521


In [250]:
def one_hot(index_array):
    b = np.zeros((index_array.size, index_array.max()+1))
    b[np.arange(index_array.size),index_array] = 1

    return b

def brier_score(preds, true):
    sum_score = 0
    for i in range(len(preds)):
        p = preds[i]
        t = true[i]
        sum_score += np.sum([(p[x]-t[x])**2 for x in range(len(p))])
    return sum_score/len(preds)
  
def rps(preds, true):
    rps = np.zeros(len(preds))

    for x in range(len(preds)):
      p = preds[x]
      t = true[x]
      cumulative = 0
      for i in range(1, len(preds[0])):
        cumulative += (sum(p[:i]) - sum(t[:i])) ** 2
        rps[x] = cumulative / (len(preds[0])-1)

    return np.mean(rps)


print(rps(preds, one_hot(test_results)))
print(rps(test_odds, one_hot(test_results)))
print(len(test_results[test_results == 2]))

0.2732942995758766
0.19889276859504132


72